In [ ]:
import os

os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["DISABLE_TQDM"] = "1"

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

# https://civitai.com/models/20330/three-delicacy-wonton-mix?modelVersionId=24173
# wget --content-disposition https://civitai.com/api/download/models/24173
# checkpoint 파일 경로
checkpoint_path = "/home/ubuntu/work/threeDelicacyWonton_v10.safetensors"

# pipeline 생성
pipe = StableDiffusionPipeline.from_single_file(
    checkpoint_path,
    torch_dtype=torch.float16,
    use_safetensors=True
)

# GPU로 모델 이동 (CUDA를 사용할 수 있는 경우)
pipe = pipe.to("cuda")

# 테스트 이미지 생성
prompt = "A beautiful girl with mountains and a lake"
image = pipe(prompt).images[0]


In [ ]:
image

In [ ]:
prompt="masterpiece, best quality, an extremely delicate and beautiful girl,an extremely delicate and beautiful, world masterpiece theater, ultra-detailed, highly detailed, best quality, blonde hair, highres, extremely detailed,1girl, best quality, illustration, looking at viewer, impasto, canvas, oil painting, realistic, realist ,real,"
negative_prompt = "(worst quality, low quality:1.4), (bad_prompt_version2:0.8), EasyNegative, 3D, render, lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry, bad feet,"
image = pipe(prompt, negative_prompt=negative_prompt).images[0]
image


In [ ]:
# https://civitai.com/models/6020/zelda-cdi-style-lora?modelVersionId=7018
# https://civitai.com/models/6020/zelda-cdi-style-lora?modelVersionId=7018

pipe.load_lora_weights("/home/ubuntu/work/ZeldaCDI256NAI.safetensors", adapter_name="zelda")

In [ ]:
prompt = "A beautiful girl in red dress <lora:ZeldaCDI256NAI:1>"

lora_scale = 1.2
image = pipe(
    prompt, num_inference_steps=100, cross_attention_kwargs={"scale": lora_scale}, generator=torch.manual_seed(0)
).images[0]
image

In [ ]:
pipe.disable_lora()

prompt = "A beautiful girl in red dress"

image = pipe(
    prompt, num_inference_steps=100, generator=torch.manual_seed(0)
).images[0]
image

In [ ]:
pipe.enable_lora()  # enable lora again, after we disabled it above
prompt = "A beautiful girl in red dress <lora:ZeldaCDI256NAI:1>"
adapter_weight_scales = { "unet": { "down": 1, "mid": 0, "up": 0} }
pipe.set_adapters("zelda", adapter_weight_scales)
image = pipe(prompt, num_inference_steps=100, generator=torch.manual_seed(0)).images[0]
image

In [ ]:
adapter_weight_scales = { "unet": { "down": 0, "mid": 1, "up": 0} }
pipe.set_adapters("zelda", adapter_weight_scales)
image = pipe(prompt, num_inference_steps=100, generator=torch.manual_seed(0)).images[0]
image

In [ ]:
adapter_weight_scales = { "unet": { "down": 0, "mid": 0, "up": 1} }
pipe.set_adapters("zelda", adapter_weight_scales)
image = pipe(prompt, num_inference_steps=100, generator=torch.manual_seed(0)).images[0]
image

In [ ]:
active_adapters = pipe.get_active_adapters()
active_adapters

In [ ]:
list_adapters_component_wise = pipe.get_list_adapters()
list_adapters_component_wise